<h3>How to get the latest version id for a given software package from astro-update</h3>


In [1]:
%pylab inline
from bs4 import BeautifulSoup
import urllib2
import subprocess


Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
response = urllib2.urlopen('http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/')
html=response.read()

In [3]:
soup=BeautifulSoup(''.join(html))

In [4]:
table=soup.findAll('table')
soft_table=table[1] # there are 3 tables on the page, the software version table is the 2nd table

In [5]:
rows = soft_table.findAll('tr')
i=0
name=[]
for row in rows[1:]:
    cols = row.findAll('td')
    i=i+1
    n   =''.join(cols[0].find(text=True))
    name.append(n)
    rep    =''.join(cols[1].find(text=True))
    vers   =''.join(cols[2].find(text=True))
    moddate=''.join(cols[3].find(text=True))
    print "%i %s was last updated by %s to version %s on %s" % (i,n, rep, vers, moddate)
#    progname=''.join(td.find(text=True))
#    print progname

1 XSPEC was last updated by HEASARC to version 12.9.0b on 08/07/2015
2 SPOT/LEOPARD was last updated by SSC to version 19.0  on 08/05/2015
3 HEASoft was last updated by HEASARC to version  6.17 on 07/29/2015
4 ACIS_EXTRACT was last updated by PSU to version  2015Jul7 on 07/07/2015
5 SAE was last updated by FSSC to version  v10r0p5 on 06/24/2015
6 MARX was last updated by CXC/MIT to version 5.1 on 04/28/2015
7 Skyview was last updated by HEASARC to version 3.1.12 on 04/22/2015
8 CPL was last updated by ESO to version 6.6 on 03/06/2015
9 GASGANO was last updated by ESO to version 2.4.8 on 01/08/2015
10 PIMMS was last updated by HEASARC to version  4.7d on 12/24/2014
11 CIAO was last updated by CXC to version  4.7  on 12/16/2014
12 XSA was last updated by XMM-SOC to version  8.3 on 12/04/2014
13 SAS was last updated by XMM-SOC to version  14.0.0 on 11/12/2014
14 IDL Astronomical Users Library was last updated by ASD/GSFC to version 02 Nov 2014 on 11/02/2014
15 APT was last updated by STSc

In [13]:
if 'FV' in name:
    print "CPL in name"
else:
    print "missing"

missing


In [38]:
def astroupdate_dict2(url="http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/"):
    from bs4 import BeautifulSoup
    import urllib2
    response = urllib2.urlopen('http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/')
    html=response.read()
    soup=BeautifulSoup(''.join(html))
    table=soup.findAll('table')
    soft_table=table[1] # there are 3 tables on the page, the software version table is the 2nd table
    rows = soft_table.findAll('tr')
    au_dict=dict()
    for row in rows[1:]:
        cols = row.findAll('td')
        n   =''.join(cols[0].find(text=True))
        r   =''.join(cols[1].find(text=True))
        v   =''.join(cols[2].find(text=True))
        m   =''.join(cols[3].find(text=True))
        au_dict[n]=(v,m,r)
    return au_dict

def astroupdate_dict(url="http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/"):
    from bs4 import BeautifulSoup
    import urllib2
    response = urllib2.urlopen('http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/')
    html=response.read()
    soup=BeautifulSoup(''.join(html))
    table=soup.findAll('table')
    soft_table=table[1] # there are 3 tables on the page, the software version table is the 2nd table
    rows = soft_table.findAll('tr')
    au_dict=dict()
    for row in rows[1:]:
        cols = row.findAll('td')
        n   =''.join(cols[0].find(text=True))
        r   =''.join(cols[1].find(text=True))
        v   =''.join(cols[2].find(text=True))
        m   =''.join(cols[3].find(text=True))
        au_dict[str(n).lower().strip()]={'Version':str(v),'Date':str(m),'Author':str(r)}
    return au_dict
        
        
def astroupdate(software, chatter=0):
    aud=astroupdate_dict()
    softkey=software.strip().lower()
    try:
        aud[softkey]
        if chatter > 0:
            print "{0} was last updated to version {1} on {2} by {3}".format(software, 
                                                                         aud[software]['Version'], 
                                                                         aud[software]['Date'], 
                                                                         aud[software]['Author'])
        ver = str(aud[softkey]['Version']) 
        date = str(aud[softkey]['Date'])
        author = str(aud[softkey]['Author'])
        return ver, date, author   
    except KeyError:
        print "%s not monitored by Astro-Update" % software
        print "Valid entries are:"
        print aud.keys()

        
def auto_update(software):
    import subprocess
    import os
    import glob
    import webbrowser
    current_vers, moddate, resp = astroupdate(software.strip().lower())

    if software=="HEASoft":    
        fver_installed = subprocess.check_output(['fversion'])
        vers=fver_installed.strip("\n").split('_V')
        vers=vers[1].rstrip()
        print "Latest version of  HEASoft = %s; You currently have HEASoft version %s" % (current_vers, vers)
        if current_vers.strip()<> vers.strip():
            ans=''
            ans=raw_input("Would you like to update (Y/n)? ")
            if ans.strip()=='' or ans[0].lower()=='y':
                print "Opening HEASoft download page in your web browser"
                webbrowser.open('http://heasarc.gsfc.nasa.gov/docs/software/lheasoft/download.html')            
            
    if software=='SAE':
        fermidir=os.getenv('FERMI_DIR')
        if not fermidir:
            print "Environment variable $FERMI_DIR is not defined; stopping"
            return
        STools= glob.glob(fermidir+'/../../ScienceTools*') # Find Science Tools directory
        #print STools
        if not STools:
            print "Problem finding {0}".format(fermidir+'/../..')
            return
        SToolsver=STools[0].split('Tools-')[1].split('-fssc')[0]
        #print SToolsver
        print "The current version of the Science Analysis Environment for Fermi is {0}; you have version {1}".format(current_vers.strip(), SToolsver.strip())
        if current_vers<>SToolsver:
            ans=''
            ans=raw_input("Would you like to update (Y/n)? ")
            if ans.strip()=='' or ans[0].lower()=='y':
                print "Opening Fermi SAE download page in your web browser"
                webbrowser.open('http://fermi.gsfc.nasa.gov/ssc/data/analysis/software/')
    return
                
                          


In [39]:
auto_update('HEASoft')

Latest version of  HEASoft =  6.17; You currently have HEASoft version 6.17


In [7]:
ver, date, auth = astroupdate('HEASoft')

HEASoft not monitored by Astro-Update
Valid entries are:
['cfitsio', 'xspec 11', 'saoimage ds9', 'ciao', 'pros', 'mopex', 'simx', 'pyraf', 'sae', 'ccfits', 'gasgano', 'astrogrid vo desktop', 'starview', 'osa', 'tempo2', 'galprop', 'tlusty', 'synspec', 'heasoft', 'spex', 'sherpa', 'wcslib', 'casa', 'tara', 'visit', 'xronos', 'grace', 'vlt instrument pipelines', 'sas', 'splash', 'skyview', 'sextractor', 'marx', 'aips', 'exsas', 'ureka', 'pandora', 'fv', 'iraf', 'aladin', 'pimms', 'chianti idl package', 'web hera', 'ximage', 'astropy', 'apt', 'window to the stars', 'isis', 'xstar', 'browse_extract.pl', 'scisoft', 'gaia and skycat', 'cpl', 'acis_extract', 'spot/leopard', 'eclipse', 'browse_extract_wget.pl', 'stsdas', 'gsl', 'profit', 'xsa', 'xspec', 'stsci python', 'p2pp', 'midas', 'w3browse', 'idl astronomical users library', 'starlink', 'pintofale']


TypeError: 'NoneType' object is not iterable

In [31]:
software='fv'
vers, modd, resp = astroupdate(software)
vers

fv was last updated to version  5.3 on 07/16/2009 by HEASARC


'5.3'

In [12]:
fver_installed = subprocess.check_output(['fversion'])
a=fver_installed.split("_V")
b=a[1].rstrip()
b

'6.17'

In [11]:
auto_update("HEASoft")

heasoft
heasoft was last updated to version  6.17 on 07/29/2015 by HEASARC
Latest version of  HEASoft =  6.17; You currently have Version 6.17


In [25]:
aud=astroupdate_dict2()
key='heasoft'
try:
    aud[key]
    print aud[key]
except KeyError:
    print "%s not in dict" % key

{'Updated': '07/29/2015', 'Version': ' 6.17', 'By': 'HEASARC'}


In [30]:
hver=aud['heasoft']['Version']
hver

' 6.17'

In [36]:
type((name))

list

In [159]:
import subprocess
fver_output = subprocess.check_output(['fversion'])

In [166]:
fver_output.strip("\n").split('_V')

['24Jan2014', '6.15.1']

In [227]:
print k

[u'GAIA AND SKYCAT', u'XSTAR', u'CPL', u'PANDORA', u'ACIS_EXTRACT', u'Sherpa', u'CIAO', u'SPOT/LEOPARD', u'TLUSTY', u'TEMPO2', u'VLT Instrument Pipelines', u'browse_extract_wget.pl', u'VISIT', u'STSDAS', u'P2PP', u'STARLINK', u'GSL', u'STScI Python', u'UKIRT-OT', u'Ureka', u'ISIS', u'HEASoft', u'SEXTRACTOR', u'PROFIT', u'PIMMS', u'SAE', u'GASGANO', u'SPLASH', u'PINTofALE', u'SAS', u'CMBEASY', u'GRACE', u'MOPEX', u'STARVIEW', u'SCISOFT', u'SAOImage ds9', u'Aladin', u'fv', u'IRAF', u'CHIANTI IDL package', u'EXSAS', u'CCfits', u'OSA', u'ASTROGRID VO DESKTOP', u'XSPEC', u'WINDOW TO THE STARS', u'MIDAS', u'CASA', u'browse_extract.pl', u'CFITSIO', u'XSPEC 11', u'SPEX', u'XIMAGE', u'WCSLIB', u'IDL Astronomical Users Library', u'PROS', u'XRONOS', u'APT', u'AIPS', u'ASTROPY', u'SYNSPEC', u'W3Browse', u'ECLIPSE', u'GALPROP', u'TARA']


In [228]:
print len(k)

65


In [229]:
print k[64]

TARA


In [242]:
arange(0,len(k)-1,9)

array([ 0,  9, 18, 27, 36, 45, 54, 63])

<hr>
<h2>How to find the latest version for a CALDB</h2>

we define a caldb python dictionary as follows:

caldb={'ASCA':{'GIS':20060711,'SIS':19991111,'XRT':19991111,'MIS':19940114}, 'BEPPOSAX':{'HGSPC':1999, 'LECS':1999, 'MECS':1999, 'PDS':1999, 'WFC':1999} ...}

this dictionary should be defined by parsing the "supported missions" webpage
http://heasarc.gsfc.nasa.gov/docs/heasarc/caldb/caldb_supported_missions.html

In [ ]:
def caldb_dict(url="http://heasarc.gsfc.nasa.gov/docs/heasarc/caldb/caldb_supported_missions.html"):
    """
    Parse the caldb supported missions web page and convert to a python library
    """
    from bs4 import BeautifulSoup
    import urllib2
    response = urllib2.urlopen(url)
    html=response.read()
    soup=BeautifulSoup(''.join(html))
    table=soup.findAll('table')
    hcal_table=table[1] # there are 3 tables on the page, the caldb version table is the 2nd table
    rows = hcal_table.findAll('tr')
    #a= ''.join(cols.find(text=True)).strip()
    #print a
    caldb_dict=dict()
    """
    Read first row - contains the first mission
       if first cell in row is not empty: mission name=mis
    """
    cols = rows[0].findAll('td')
    mis =str(cols[0].find(text=True)).strip().lower()
    #print "Found %s" % mis
    """
    Read next row
       check that first cell in row is empty: 
         if cell empty, then add to instrument dictionary
            inst_dict[cell[1]]=cell[2] (i.e. something like inst_dict={'XIS':'20140101'}) 
                 where cell[1] is the instrument name (XIS) and cell[2] is the version date 
          if cell is not empty: found new mission
               so store instr_dict in caldb_dict for previous mission: CALDB_DICT[MIS]=inst_dict
               set mis = new mission
               clear inst_dict
               repeat
    """
    inst_dict=dict()
    for row in rows[1:]: # omit first row (i.e. row 0)
        cols = row.findAll('td')
        #print cols[0]
        a=cols[0].find(text=True)
        #print a
        if len(a.strip()) == 0: # no entry in mission cell, so these are instruments
            inst= str(cols[1].find(text=True)).strip().lower()
            vers= str(cols[2].find(text=True)).strip()
            #print mis, inst, vers
            try:
                inst_dict[inst]=caldate2vers(vers)
            except:
                inst_dict[inst]=vers
        else: # save instruments & version in mission dictionary
            #print inst_dict
            #print "current mission is %s" % mis
            caldb_dict[mis]=inst_dict
            #print caldb_dict
            inst_dict=dict()
            mis=str(a.strip()).lower()
            #print "Found new mission %s " % mis
    return caldb_dict

def caldate2vers(update_date):
    """
    converts latest update dates (a string in mmm 00, yyyy format) to yyyymmdd format
    """
    month=({'Apr':'04','Aug':'08','Dec':'12','Feb':'02','Jan':'01','Jul':'07','Jun':'06','Mar':'03','May':'05','Nov':'11',
          'Oct':'10','Sep':'09'})
    a=update_date.split(',')
    yyyy=a[1].strip()
    b=a[0].split()
    mon=b[0]
    day=b[1]
    return yyyy+month[mon]+day
    
def current_caldb_vers(mis, inst):
    import ftplib
    caldir="caldb/data/"+mis.lower().strip()+"/"+inst.lower().strip()
    ftp = ftplib.FTP("heasarc.gsfc.nasa.gov")
    ftp.login("anonymous", "")
    ftpmess=ftp.cwd(caldir)
    list=[]
    ftp.dir('caldb.indx', list.append)
    ftp.close()
    vers=list[0].split()[-1].split('/')[1].split('indx')[-1] #[-1] is the last element
    return vers

In [86]:
caldict=caldb_dict()

In [99]:
mis='xte'
inst='pca'
print caldict[mis][inst]
cur_vers=current_caldb_vers(mis, inst)
print cur_vers
if caldict['nustar']['fpm']==cur_vers:
    print "You're Current!"
else:
    print "Not current :-("

KeyError: 'xte'

In [192]:
a=caldict['ASCA']['GIS'].split(',')
yyyy=a[1].strip()
yyyy
a[0]
a[0].split()
c='1999'+'11'+'11'
c

print caldict['ASCA']['GIS']
print caldate2vers('Nov 11, 1999')

Nov 11, 1999
19991111


In [208]:
caldict['ASCA']

{'GIS': '19991111',
 'MIS': '19940114',
 'SIS': 'July 11, 2006',
 'XRT': '19991111'}

In [57]:
a={'b':'c',2:5}
a
a.clear()
a

{}

In [107]:
from bs4 import BeautifulSoup
import urllib2
url="http://heasarc.gsfc.nasa.gov/docs/heasarc/caldb/caldb_supported_missions.html"
response = urllib2.urlopen(url)
html=response.read()
soup = BeautifulSoup(html)
table=soup.findAll('table')
hcal_table=table[1]# there are 3 tables on the page, the software version table is the 2nd table
rows = hcal_table.findAll('tr')
cols = rows[3].findAll('td')
mis =str(cols[0].find(text=True))

In [179]:
import calendar
enumerate(calendar.month_abbr)
dict((v,k) for k,v in enumerate(calendar.month_abbr))

{'': 0,
 'Apr': 4,
 'Aug': 8,
 'Dec': 12,
 'Feb': 2,
 'Jan': 1,
 'Jul': 7,
 'Jun': 6,
 'Mar': 3,
 'May': 5,
 'Nov': 11,
 'Oct': 10,
 'Sep': 9}

In [181]:
mon['Sep']

'09'

In [82]:
import ftplib
ftp = ftplib.FTP("heasarc.gsfc.nasa.gov")
ftp.login("anonymous", "")
ftpmess=ftp.cwd("caldb/data/nustar/fpm")
list=[]
ftp.dir('caldb.indx', list.append)
ftp.close()
list

['lrwxrwxrwx   1 1389     340            24 Jan 17 12:31 caldb.indx -> index/caldb.indx20131223']

In [ ]:
a=list[0].split()

In [ ]:
a[-1]

In [83]:
vers=list[0].split()[-1].split('/')[1].split('indx')[-1]

In [84]:
vers

'20131223'

In [59]:
a='ABC'

In [61]:
a.lower()

'abc'